In [95]:
import pandas as pd
import numpy as np

In [97]:

file_path = "/Users/sandyajaleshkumar/Desktop/Data Analytics & Data Mining/Assignment/Datasets/fraudTest.csv"
df = pd.read_csv(file_path)

In [98]:
df_info = df.info()
df_info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             555719 non-null  int64  
 1   trans_date_trans_time  555719 non-null  object 
 2   cc_num                 555719 non-null  int64  
 3   merchant               555719 non-null  object 
 4   category               555719 non-null  object 
 5   amt                    555719 non-null  float64
 6   first                  555719 non-null  object 
 7   last                   555719 non-null  object 
 8   gender                 555719 non-null  object 
 9   street                 555719 non-null  object 
 10  city                   555719 non-null  object 
 11  state                  555719 non-null  object 
 12  zip                    555719 non-null  int64  
 13  lat                    555719 non-null  float64
 14  long                   555719 non-nu

In [99]:
df_head = df.head()
df_head


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [100]:
df_shape = df.shape
df_shape

(555719, 23)

In [101]:
df.drop_duplicates(inplace=True)

In [102]:
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [103]:

columns = df.columns
class_distribution = df['is_fraud'].value_counts(normalize=True) * 100

columns, class_distribution

(Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
        'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
        'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
        'merch_lat', 'merch_long', 'is_fraud'],
       dtype='object'),
 is_fraud
 0    99.614014
 1     0.385986
 Name: proportion, dtype: float64)

Calculates rolling 7-day transaction count and amount sum for each card based on transaction timestamps.

In [106]:

df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

df.sort_values(by=['cc_num', 'trans_date_trans_time'], inplace=True)

df['rolling_txn_count_7d'] = 0
df['rolling_amt_sum_7d'] = 0.0

results = []

for cc, group in df.groupby('cc_num'):
    group = group.set_index('trans_date_trans_time')
    group = group.sort_index()
    group['rolling_txn_count_7d'] = group['amt'].rolling('7d').count()
    group['rolling_amt_sum_7d'] = group['amt'].rolling('7d').sum()
    results.append(group)
    
df_rolled = pd.concat(results).reset_index()

df = df_rolled

df[['cc_num', 'trans_date_trans_time', 'rolling_txn_count_7d', 'rolling_amt_sum_7d']].head()

,cc_num,trans_date_trans_time,rolling_txn_count_7d,rolling_amt_sum_7d
0,60416207185,2020-06-21 13:05:42,1.0,124.66
1,60416207185,2020-06-21 16:25:36,2.0,203.18
2,60416207185,2020-06-22 07:58:33,3.0,268.43
3,60416207185,2020-06-22 15:32:31,4.0,356.17
4,60416207185,2020-06-23 12:28:54,5.0,504.19


Extracts date and time features from transaction timestamps and calculates the number of transactions per card per day, week, and month.

In [111]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['trans_date'] = df['trans_date_trans_time'].dt.date

df['hour'] = df['trans_date_trans_time'].dt.hour
df['day'] = df['trans_date_trans_time'].dt.day
df['day_of_week'] = df['trans_date_trans_time'].dt.dayofweek
df['week'] = df['trans_date_trans_time'].dt.strftime('%Y-%U')  # year-week number
df['month'] = df['trans_date_trans_time'].dt.to_period('M')

daily_count = df.groupby(['cc_num', 'trans_date']).size().reset_index(name='txn_count_per_day')
weekly_count = df.groupby(['cc_num', 'week']).size().reset_index(name='txn_count_per_week')
monthly_count = df.groupby(['cc_num', 'month']).size().reset_index(name='txn_count_per_month')


df = df.merge(daily_count, on=['cc_num', 'trans_date'], how='left')
df = df.merge(weekly_count, on=['cc_num', 'week'], how='left')
df = df.merge(monthly_count, on=['cc_num', 'month'], how='left')


df[['cc_num', 'trans_date', 'week', 'month', 'txn_count_per_day', 'txn_count_per_week', 'txn_count_per_month']].head()


,cc_num,trans_date,week,month,txn_count_per_day,txn_count_per_week,txn_count_per_month
0,60416207185,2020-06-21,2020-25,2020-06,2,21,34
1,60416207185,2020-06-21,2020-25,2020-06,2,21,34
2,60416207185,2020-06-22,2020-25,2020-06,2,21,34
3,60416207185,2020-06-22,2020-25,2020-06,2,21,34
4,60416207185,2020-06-23,2020-25,2020-06,5,21,34


Converts date of birth to datetime and calculates the age of the cardholder at the time of the transaction.

In [118]:
df['dob'] = pd.to_datetime(df['dob'])
df['age'] = df['trans_date_trans_time'].dt.year - df['dob'].dt.year
df


,trans_date_trans_time,Unnamed: 0,cc_num,merchant,category,amt,first,last,gender,street,...,trans_date,hour,day,day_of_week,week,month,txn_count_per_day,txn_count_per_week,txn_count_per_month,age
0,2020-06-21 13:05:42,157,60416207185,fraud_Kutch-Ferry,home,124.66,Mary,Diaz,F,9886 Anita Drive,...,2020-06-21,13,21,6,2020-25,2020-06,2,21,34,34
1,2020-06-21 16:25:36,741,60416207185,fraud_Halvorson Group,misc_pos,78.52,Mary,Diaz,F,9886 Anita Drive,...,2020-06-21,16,21,6,2020-25,2020-06,2,21,34,34
2,2020-06-22 07:58:33,3047,60416207185,fraud_Conroy-Cruickshank,gas_transport,65.25,Mary,Diaz,F,9886 Anita Drive,...,2020-06-22,7,22,0,2020-25,2020-06,2,21,34,34
3,2020-06-22 15:32:31,4351,60416207185,fraud_Larkin Ltd,kids_pets,87.74,Mary,Diaz,F,9886 Anita Drive,...,2020-06-22,15,22,0,2020-25,2020-06,2,21,34,34
4,2020-06-23 12:28:54,7695,60416207185,fraud_Leffler-Goldner,personal_care,148.02,Mary,Diaz,F,9886 Anita Drive,...,2020-06-23,12,23,1,2020-25,2020-06,5,21,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 02:10:10,552584,4992346398065154184,fraud_Parisian and Sons,gas_transport,66.11,Benjamin,Kim,M,920 Patrick Light,...,2020-12-31,2,31,3,2020-52,2020-12,5,42,189,64
555715,2020-12-31 05:13:35,552892,4992346398065154184,fraud_Bogisich-Homenick,misc_net,4.58,Benjamin,Kim,M,920 Patrick Light,...,2020-12-31,5,31,3,2020-52,2020-12,5,42,189,64
555716,2020-12-31 11:15:44,553559,4992346398065154184,fraud_Erdman-Kertzmann,gas_transport,95.96,Benjamin,Kim,M,920 Patrick Light,...,2020-12-31,11,31,3,2020-52,2020-12,5,42,189,64
555717,2020-12-31 11:17:25,553560,4992346398065154184,fraud_Stracke-Lemke,grocery_pos,149.48,Benjamin,Kim,M,920 Patrick Light,...,2020-12-31,11,31,3,2020-52,2020-12,5,42,189,64


In [119]:
Creates new features: whether the transaction was on a weekend or at night, 
and calculates the distance between user and merchant using the Haversine formula.


In [120]:
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

df['is_night'] = df['hour'].between(0, 6).astype(int)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371 
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

df['distance_to_merchant'] = haversine(df['lat'], df['long'], df['merch_lat'], df['merch_long'])

df[['hour', 'day_of_week', 'is_weekend', 'is_night', 'distance_to_merchant']].head()


,hour,day_of_week,is_weekend,is_night,distance_to_merchant
0,13,6,1,0,30.457093
1,16,6,1,0,91.942935
2,7,0,0,0,121.857420
3,15,0,0,0,65.414164
4,12,1,0,0,38.311273


Calculates average and standard deviation of transaction amounts per card, 
then computes the z-score to measure how unusual each transaction is.


In [122]:
card_stats = df.groupby('cc_num')['amt'].agg(
    avg_amt_per_card='mean',
    std_amt_per_card='std'
).reset_index()

df = df.merge(card_stats, on='cc_num', how='left')

epsilon = 1e-3
df['std_amt_per_card'].replace(0, epsilon, inplace=True)

df['z_score_amt_per_card'] = (df['amt'] - df['avg_amt_per_card']) / df['std_amt_per_card']

df[['cc_num', 'amt', 'avg_amt_per_card', 'std_amt_per_card', 'z_score_amt_per_card']].head()


/var/folders/x0/50_cl34x1zg9wnnjkygh0svc0000gn/T/ipykernel_54620/923703010.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['std_amt_per_card'].replace(0, epsilon, inplace=True)


,cc_num,amt,avg_amt_per_card,std_amt_per_card,z_score_amt_per_card
0,60416207185,124.66,66.499484,180.015519,0.323086
1,60416207185,78.52,66.499484,180.015519,0.066775
2,60416207185,65.25,66.499484,180.015519,-0.006941
3,60416207185,87.74,66.499484,180.015519,0.117993
4,60416207185,148.02,66.499484,180.015519,0.452853


Calculates time since the last transaction for each card and adds the total number of transactions per merchant.

In [126]:

df.sort_values(by=['cc_num', 'trans_date_trans_time'], inplace=True)
df['time_since_last_txn'] = df.groupby('cc_num')['trans_date_trans_time'].diff().dt.total_seconds().fillna(0)

merchant_txn_count = df['merchant'].value_counts().to_dict()
df['merchant_txn_count'] = df['merchant'].map(merchant_txn_count)

Categorizes transaction amounts into buckets and adds fraud rate by state and job as new features.

In [129]:

df['amt_bucket'] = pd.cut(
    df['amt'],
    bins=[-1, 10, 100, 500, 1000, 5000, float('inf')],
    labels=['very_low', 'low', 'medium', 'high', 'very_high', 'extreme']
)

state_fraud_rate = df.groupby('state')['is_fraud'].mean().to_dict()
df['state_fraud_rate'] = df['state'].map(state_fraud_rate)

job_fraud_rate = df.groupby('job')['is_fraud'].mean().to_dict()
df['job_fraud_rate'] = df['job'].map(job_fraud_rate)


Calculates rolling 7-day transaction count and amount sum for each card based on transaction timestamps.

In [132]:

df['trans_date'] = pd.to_datetime(df['trans_date'])

df.sort_values(by=['cc_num', 'trans_date'], inplace=True)

df['rolling_txn_count_7d'] = 0
df['rolling_amt_sum_7d'] = 0.0

results = []

for cc, group in df.groupby('cc_num'):
    group = group.set_index('trans_date')
    group = group.sort_index()
    group['rolling_txn_count_7d'] = group['amt'].rolling('7d').count()
    group['rolling_amt_sum_7d'] = group['amt'].rolling('7d').sum()
    results.append(group)

df_rolled = pd.concat(results).reset_index()

df = df_rolled

df[['cc_num', 'trans_date', 'rolling_txn_count_7d', 'rolling_amt_sum_7d']].head()

,cc_num,trans_date,rolling_txn_count_7d,rolling_amt_sum_7d
0,60416207185,2020-06-21,1.0,124.66
1,60416207185,2020-06-21,2.0,203.18
2,60416207185,2020-06-22,3.0,268.43
3,60416207185,2020-06-22,4.0,356.17
4,60416207185,2020-06-23,5.0,504.19


In [134]:
monthly_group = df.groupby(['cc_num', 'month'])['amt']
df['monthly_avg_amt'] = monthly_group.transform('mean')
df['monthly_std_amt'] = monthly_group.transform('std')

df['monthly_z_score_amt'] = (df['amt'] - df['monthly_avg_amt']) / df['monthly_std_amt']
df['monthly_z_score_amt'] = df['monthly_z_score_amt'].fillna(0)

df[['cc_num', 'trans_date_trans_time', 'amt', 'monthly_avg_amt', 'monthly_std_amt', 'monthly_z_score_amt']].head()

,cc_num,trans_date_trans_time,amt,monthly_avg_amt,monthly_std_amt,monthly_z_score_amt
0,60416207185,2020-06-21 13:05:42,124.66,59.618235,42.482769,1.531015
1,60416207185,2020-06-21 16:25:36,78.52,59.618235,42.482769,0.444928
2,60416207185,2020-06-22 07:58:33,65.25,59.618235,42.482769,0.132566
3,60416207185,2020-06-22 15:32:31,87.74,59.618235,42.482769,0.661957
4,60416207185,2020-06-23 12:28:54,148.02,59.618235,42.482769,2.080885


In [137]:
columns_to_drop = [
    'Unnamed: 0', 'first', 'last', 'street', 'city', 'zip', 'dob', 'trans_num', 'unix_time',
    'lat', 'long', 'merch_lat', 'merch_long', 'trans_date', 'week', 'month'
]

df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)


In [145]:
columns_to_drop = [
    'Unnamed: 0', 'first', 'last', 'street', 'city', 'zip', 'dob',
    'trans_num', 'unix_time', 'lat', 'long', 'merch_lat', 'merch_long',
    'trans_date', 'week', 'month'
]

ndf = df.drop(columns=[col for col in columns_to_drop if col in df.columns]).copy()

Encoding

In [153]:
ndf.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   trans_date_trans_time  555719 non-null  datetime64[ns]
 1   cc_num                 555719 non-null  int64         
 2   merchant               555719 non-null  object        
 3   category               555719 non-null  object        
 4   amt                    555719 non-null  float64       
 5   gender                 555719 non-null  object        
 6   state                  555719 non-null  object        
 7   city_pop               555719 non-null  int64         
 8   job                    555719 non-null  object        
 9   is_fraud               555719 non-null  int64         
 10  rolling_txn_count_7d   555719 non-null  float64       
 11  rolling_amt_sum_7d     555719 non-null  float64       
 12  hour                   555719 non-null  int3

In [154]:

columns_to_drop = [
    'cc_num', 'trans_date_trans_time',
    'std_amt_per_card', 'avg_amt_per_card',
    'monthly_avg_amt', 'monthly_std_amt'
]

ndf_encoded = ndf.drop(columns=columns_to_drop)


In [155]:
ndf_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   merchant              555719 non-null  object  
 1   category              555719 non-null  object  
 2   amt                   555719 non-null  float64 
 3   gender                555719 non-null  object  
 4   state                 555719 non-null  object  
 5   city_pop              555719 non-null  int64   
 6   job                   555719 non-null  object  
 7   is_fraud              555719 non-null  int64   
 8   rolling_txn_count_7d  555719 non-null  float64 
 9   rolling_amt_sum_7d    555719 non-null  float64 
 10  hour                  555719 non-null  int32   
 11  day                   555719 non-null  int32   
 12  day_of_week           555719 non-null  int32   
 13  txn_count_per_day     555719 non-null  int64   
 14  txn_count_per_week    555719 non-nul

In [156]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

for col in ['merchant', 'job', 'state']:
    freq_encoding = ndf_encoded[col].value_counts().to_dict()
    ndf_encoded[col + '_freq'] = ndf_encoded[col].map(freq_encoding)




In [157]:
ndf_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   merchant              555719 non-null  object  
 1   category              555719 non-null  object  
 2   amt                   555719 non-null  float64 
 3   gender                555719 non-null  object  
 4   state                 555719 non-null  object  
 5   city_pop              555719 non-null  int64   
 6   job                   555719 non-null  object  
 7   is_fraud              555719 non-null  int64   
 8   rolling_txn_count_7d  555719 non-null  float64 
 9   rolling_amt_sum_7d    555719 non-null  float64 
 10  hour                  555719 non-null  int32   
 11  day                   555719 non-null  int32   
 12  day_of_week           555719 non-null  int32   
 13  txn_count_per_day     555719 non-null  int64   
 14  txn_count_per_week    555719 non-nul

In [158]:
ndf_encoded = pd.get_dummies(ndf_encoded, columns=['category'], prefix='cat')

In [159]:
le = LabelEncoder()
ndf_encoded['gender_encoded'] = le.fit_transform(ndf_encoded['gender'])

ordinal_mapping = {cat: idx for idx, cat in enumerate(sorted(ndf_encoded['amt_bucket'].unique()))}
ndf_encoded['amt_bucket_encoded'] = ndf_encoded['amt_bucket'].map(ordinal_mapping)


ndf_encoded.drop(columns=['merchant', 'job', 'state', 'gender', 'amt_bucket'], inplace=True)


In [160]:
ndf_encoded

,amt,city_pop,is_fraud,rolling_txn_count_7d,rolling_amt_sum_7d,hour,day,day_of_week,txn_count_per_day,txn_count_per_week,...,cat_home,cat_kids_pets,cat_misc_net,cat_misc_pos,cat_personal_care,cat_shopping_net,cat_shopping_pos,cat_travel,gender_encoded,amt_bucket_encoded
0,124.66,1645,0,1.0,124.66,13,21,6,2,21,...,True,False,False,False,False,False,False,False,0,3
1,78.52,1645,0,2.0,203.18,16,21,6,2,21,...,False,False,False,True,False,False,False,False,0,2
2,65.25,1645,0,3.0,268.43,7,22,0,2,21,...,False,False,False,False,False,False,False,False,0,2
3,87.74,1645,0,4.0,356.17,15,22,0,2,21,...,False,True,False,False,False,False,False,False,0,2
4,148.02,1645,0,5.0,504.19,12,23,1,5,21,...,False,False,False,False,True,False,False,False,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,66.11,532,0,46.0,4558.64,2,31,3,5,42,...,False,False,False,False,False,False,False,False,1,2
555715,4.58,532,0,47.0,4563.22,5,31,3,5,42,...,False,False,True,False,False,False,False,False,1,5
555716,95.96,532,0,48.0,4659.18,11,31,3,5,42,...,False,False,False,False,False,False,False,False,1,2
555717,149.48,532,0,49.0,4808.66,11,31,3,5,42,...,False,False,False,False,False,False,False,False,1,3


In [164]:
ndf_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 40 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   amt                   555719 non-null  float64 
 1   city_pop              555719 non-null  int64   
 2   is_fraud              555719 non-null  int64   
 3   rolling_txn_count_7d  555719 non-null  float64 
 4   rolling_amt_sum_7d    555719 non-null  float64 
 5   hour                  555719 non-null  int32   
 6   day                   555719 non-null  int32   
 7   day_of_week           555719 non-null  int32   
 8   txn_count_per_day     555719 non-null  int64   
 9   txn_count_per_week    555719 non-null  int64   
 10  txn_count_per_month   555719 non-null  int64   
 11  age                   555719 non-null  int32   
 12  is_weekend            555719 non-null  int64   
 13  is_night              555719 non-null  int64   
 14  distance_to_merchant  555719 non-nul

In [166]:
ndf_encoded['is_fraud'] = ndf_encoded['is_fraud'].astype(int)


In [182]:
ndf_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 40 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   amt                   555719 non-null  float64 
 1   city_pop              555719 non-null  int64   
 2   is_fraud              555719 non-null  int64   
 3   rolling_txn_count_7d  555719 non-null  float64 
 4   rolling_amt_sum_7d    555719 non-null  float64 
 5   hour                  555719 non-null  int32   
 6   day                   555719 non-null  int32   
 7   day_of_week           555719 non-null  int32   
 8   txn_count_per_day     555719 non-null  int64   
 9   txn_count_per_week    555719 non-null  int64   
 10  txn_count_per_month   555719 non-null  int64   
 11  age                   555719 non-null  int32   
 12  is_weekend            555719 non-null  int64   
 13  is_night              555719 non-null  int64   
 14  distance_to_merchant  555719 non-nul

In [186]:
X_test = ndf_encoded.drop(columns=['is_fraud'], errors='ignore')
y_test = ndf_encoded['is_fraud'].astype(int)

X_test = X_test.astype({col: int for col in X_test.select_dtypes('bool').columns})

X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

X_test.dropna(inplace=True)
y_test = y_test.loc[X_test.index]


In [188]:
cleaned_test_df = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)

cleaned_test_df.to_csv("/Users/sandyajaleshkumar/Desktop/Data Analytics & Data Mining/Assignment/Datasets/preprocessed_test.csv", index=False)
